### 12.1 FP-trees: an efficient way to encode a dataset

- 优点：一般要快于Apriori
- 缺点：实现比较困难，在某些数据集上性能会下降

### 12.2 Build an FP-tree

##### 12.2.1 Creating the FP-tree data structure

In [1]:
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        '''
        初始化树节点
        :param nameValue: 节点名称
        :param numOccur: 节点计数
        :param parentNode: 父节点
        '''
        self.name = nameValue
        self.count = numOccur
        self.parent = parentNode
        self.children = {}
        self.nodeLink = None
        
    def inc(self, numOccur):
        '''增加节点计数'''
        self.count += numOccur
        
    def disp(self, ind=1):
        '''打印树节点'''
        print('  ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)

In [3]:
rootNode = treeNode('pyramid', 9, None)
rootNode.children['eye'] = treeNode('eye', 13, rootNode)
rootNode.disp()
rootNode.children['phoenix'] = treeNode('phoenix', 3, rootNode)
rootNode.disp()

   pyramid   9
     eye   13
   pyramid   9
     eye   13
     phoenix   3


##### 12.2.2 Constructing the FP-tree

In [4]:
def updateHeader(nodeToTest, targetNode):
    '''
    更新头指针
    :param nodeToTest: 目标节点
    :param targetNode: 目标节点的头指针
    '''
    while nodeToTest.nodeLink is not None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [5]:
def updateTree(items, inTree, headerTable, count):
    '''
    更新树
    :param items: 满足minSup排序后的元素Key的数组
    :param inTree: 树节点
    :param headerTable: 头指针表
    :param count: 计数
    '''
    if items[0] in inTree.children: # 如果节点已经存在
        inTree.children[items[0]].inc(count) # 增加计数
    else:
        inTree.children[items[0]] = treeNode(items[0], count, inTree) # 创建新节点
        if headerTable[items[0]][1] is None: # 如果头指针表中没有该节点
            headerTable[items[0]][1] = inTree.children[items[0]] # 更新头指针表
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]]) # 更新头指针表
    if len(items) > 1: # 如果还有更多的元素
        updateTree(items[1:], inTree.children[items[0]], headerTable, count) # 递归更新树

In [6]:
def createTree(dataSet, minSup=1):
    '''
    创建FP树
    :param dataSet: 数据集
    :param minSup: 最小支持度
    :return: FP树和频繁项集头指针表
    '''
    headerTable = {}
    for trans in dataSet: # 遍历数据集
        for item in trans: # 遍历每个事务
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans] # 统计每个项的出现次数
            
    for k in list(headerTable.keys()): # 删除不满足最小支持度的项
        if headerTable[k] < minSup:
            del(headerTable[k])
            
    freqItemSet = set(headerTable.keys()) # 频繁项集
    if len(freqItemSet) == 0: # 如果没有频繁项集，返回None
        return None, None
    for k in headerTable: # 对频繁项集进行排序
        headerTable[k] = [headerTable[k], None]
    # 创建FP树
    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
                
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)] # 对频繁项集进行排序
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable # 返回FP树和频繁项集头指针表

In [7]:
def loadSimpDat():
    '''加载简单数据集'''
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't'],
               ['y', 'r', 'x', 'z', 'h'],
               ['r']]
    return simpDat

def createInitSet(dataSet):
    '''创建初始数据集'''
    initSet = {}
    for trans in dataSet:
        initSet[frozenset(trans)] = 1
    return initSet

In [10]:
simpDat = loadSimpDat() # 加载简单数据集
initSet = createInitSet(simpDat) # 创建初始数据集
print(initSet) # 打印初始数据集
myFPTree, myHeaderTab = createTree(initSet, 3) # 创建FP树
myFPTree.disp() # 打印FP树

{frozenset({'z', 'j', 'p', 'r', 'h'}): 1, frozenset({'z', 'x', 's', 'u', 'y', 'w', 'v', 't'}): 1, frozenset({'z'}): 1, frozenset({'x', 's', 'o', 'r', 'n'}): 1, frozenset({'z', 'x', 'q', 'y', 'r', 't'}): 1, frozenset({'z', 'x', 'y', 'r', 'h'}): 1, frozenset({'r'}): 1}
   Null Set   1
     z   5
       r   3
         x   2
           y   2
       x   1
         y   1
     r   2
       x   1


### 12.3 Mining frequent items from an FP-tree

##### 12.3.1 Extracting conditional pattern bases

In [11]:
def ascendTree(leafNode, prefixPath):
    '''
    向上遍历树
    :param leafNode: 叶子节点
    :param prefixPath: 前缀路径
    '''
    if leafNode.parent is not None:
        prefixPath.append(leafNode.name) # 添加节点名称到前缀路径
        ascendTree(leafNode.parent, prefixPath) # 递归向上遍历树
        
def findPrefixPath(basePat, treeNode):
    '''
    查找前缀路径
    :param basePat: 基础路径
    :param treeNode: 树节点
    :return: 前缀路径字典
    '''
    condPats = {}
    while treeNode is not None:
        prefixPath = []
        ascendTree(treeNode, prefixPath) # 向上遍历树
        if len(prefixPath) > 1: # 如果前缀路径长度大于1
            condPats[frozenset(prefixPath[1:])] = treeNode.count # 添加到前缀路径字典
        treeNode = treeNode.nodeLink # 更新树节点
    return condPats

In [13]:
print(findPrefixPath('x', myHeaderTab['x'][1])) # 查找前缀路径

{frozenset({'z'}): 1, frozenset({'r'}): 1, frozenset({'z', 'r'}): 2}


##### 12.3.2 Creating conditional FP-trees

In [14]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    '''
    挖掘FP树
    :param inTree: FP树
    :param headerTable: 头指针表
    :param minSup: 最小支持度
    :param preFix: 前缀路径
    :param freqItemList: 频繁项集列表
    '''
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])] # 对头指针表进行排序
    for basePat in bigL: # 遍历头指针表
        newFreqSet = preFix.copy() # 复制前缀路径
        newFreqSet.add(basePat) # 添加基础路径到前缀路径
        freqItemList.append(newFreqSet) # 添加到频繁项集列表
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1]) # 查找前缀路径
        myCondTree, myHead = createTree(condPattBases, minSup) # 创建条件FP树
        if myHead is not None:
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList) # 递归挖掘FP树

In [15]:
freqItems = [] # 频繁项集列表
mineTree(myFPTree, myHeaderTab, 3, set([]), freqItems) # 挖掘FP树
print(freqItems) # 打印频繁项集列表

[{'y'}, {'z', 'y'}, {'y', 'x'}, {'z', 'y', 'x'}, {'x'}, {'z', 'x'}, {'r', 'x'}, {'z'}, {'r'}, {'z', 'r'}]


### 12.5 Example: mining a clickstream from a news site

In [16]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()] # 读取数据集

initSet = createInitSet(parsedDat) # 创建初始数据集

myFPtree, myHeaderTab = createTree(initSet, 100000) # 创建FP树

myFreqItems = [] # 频繁项集列表
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqItems) # 挖掘FP树
print(myFreqItems) # 打印频繁项集列表

[{'1'}, {'1', '6'}, {'3'}, {'11', '3'}, {'11', '3', '6'}, {'3', '6'}, {'11'}, {'11', '6'}, {'6'}]
